In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [2]:
CLIENT_ID = '0VHQ2UD5XSSLMMMYGDDNEE25HDVXSXE3PHO5K0ETG5ARETOJ' # your Foursquare ID
CLIENT_SECRET = 'G4DE4JLYC55DCHBGXEXDVYEAAUKSWBWAKQCYFD2D1AMVRN5J' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0VHQ2UD5XSSLMMMYGDDNEE25HDVXSXE3PHO5K0ETG5ARETOJ
CLIENT_SECRET:G4DE4JLYC55DCHBGXEXDVYEAAUKSWBWAKQCYFD2D1AMVRN5J


In [3]:
address = 'Paris'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Paris are 48.8566101, 2.3514992.


In [4]:
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=0VHQ2UD5XSSLMMMYGDDNEE25HDVXSXE3PHO5K0ETG5ARETOJ&client_secret=G4DE4JLYC55DCHBGXEXDVYEAAUKSWBWAKQCYFD2D1AMVRN5J&ll=48.8566101,2.3514992&v=20180604&radius=500&limit=30'

In [5]:
results = requests.get(url).json()
'There are {} popular places in Paris.'.format(len(results['response']['groups'][0]['items']))

'There are 30 popular places in Paris.'

In [6]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-4bf41231e5eba59334341f90-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/plaza_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d164941735',
    'name': 'Plaza',
    'pluralName': 'Plazas',
    'primary': True,
    'shortName': 'Plaza'}],
  'id': '4bf41231e5eba59334341f90',
  'location': {'address': "Place de l'Hôtel de Ville",
   'cc': 'FR',
   'city': 'Paris',
   'country': 'France',
   'distance': 60,
   'formattedAddress': ["Place de l'Hôtel de Ville", '75004 Paris', 'France'],
   'lat': 48.85713761176484,
   'lng': 2.3516931321499746,
   'postalCode': '75004',
   'state': 'Île-de-France'},
  'name': "Place de l'Hôtel de Ville – Esplanade de la Libération",
  'photos': {'count': 0, 'groups': []}}}

In [7]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
dataframe = json_normalize(items) 

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Place de l'Hôtel de Ville – Esplanade de la Li...,Plaza,Place de l'Hôtel de Ville,FR,Paris,France,NaN,60,"[Place de l'Hôtel de Ville, 75004 Paris, France]",NaN,48.857138,2.351693,75004,Île-de-France,4bf41231e5eba59334341f90
1,Häagen-Dazs,Ice Cream Shop,1 rue d'Arcole,FR,Paris,France,NaN,218,"[1 rue d'Arcole, 75004 Paris, France]","[{'label': 'display', 'lng': 2.350124778904122...",48.854873,2.350125,75004,Île-de-France,4cb2fc20db32f04d6da9d34d
2,LUSH,Cosmetics Shop,3 rue du Renard,FR,Paris,France,NaN,131,"[3 rue du Renard, 75004 Paris, France]","[{'label': 'display', 'lng': 2.351621963273821...",48.857791,2.351622,75004,Île-de-France,4d628e33e4fe548152b8979e
3,L'Alsacien,Alsatian Restaurant,6 rue Saint-Bon,FR,Paris,France,NaN,202,"[6 rue Saint-Bon, 75004 Paris, France]","[{'label': 'display', 'lng': 2.350380611949489...",48.858275,2.350381,75004,Île-de-France,54351b59498e123ee4afc7ff
4,BHV Marais,Department Store,52 rue de Rivoli,FR,Paris,France,Rue de Rivoli,171,"[52 rue de Rivoli (Rue de Rivoli), 75004 Paris...","[{'label': 'display', 'lng': 2.353650749944108...",48.857210,2.353651,75004,Île-de-France,4b4b6a3df964a520b89a26e3
5,Maison Aleph,Pastry Shop,20 rue de la Verrerie,FR,Paris,France,NaN,260,"[20 rue de la Verrerie, 75004 Paris, France]","[{'label': 'display', 'lng': 2.354873, 'lat': ...",48.857348,2.354873,75004,Île-de-France,59247f4e0d2be70573edd834
6,Tour Saint-Jacques,Historic Site,88 rue de Rivoli,FR,Paris,France,NaN,248,"[88 rue de Rivoli, 75004 Paris, France]","[{'label': 'display', 'lng': 2.348874807357788...",48.858031,2.348875,75004,Île-de-France,4b5c7d1ff964a5205f3229e3
7,Berges de Seine – Rive droite,Pedestrian Plaza,De la passerelle Léopold-Sédar-Senghor au port...,FR,Paris,France,NaN,174,[De la passerelle Léopold-Sédar-Senghor au por...,"[{'label': 'display', 'lng': 2.352289446225441...",48.855131,2.352289,NaN,Île-de-France,58d800df9435a979b8a645fa
8,Paris Rendez-Vous,Souvenir Shop,29 rue de Rivoli,FR,Paris,France,Rue Lobau,115,"[29 rue de Rivoli (Rue Lobau), 75004 Paris, Fr...","[{'label': 'display', 'lng': 2.352841377331694...",48.857150,2.352841,75004,Île-de-France,5361075a498ea34171cb06ca
9,Hotel Duo Paris,Hotel,11 rue du Temple,FR,Paris,France,Rue de la Verrerie,193,"[11 rue du Temple (Rue de la Verrerie), 75004 ...","[{'label': 'display', 'lng': 2.352936699640952...",48.858065,2.352937,75004,Île-de-France,4e84bdeb2fb6611bdfd704ea


In [9]:
# Get the location data for each borough of Paris
address = '75000'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudezero = location.latitude
longitudezero = location.longitude
print('The geograpical coordinate of 75000 are {}, {}.'.format(latitudezero, longitudezero))

address = '75001'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudeone = location.latitude
longitudeone = location.longitude
print('The geograpical coordinate of 75001 are {}, {}.'.format(latitudeone, longitudeone))

address = '75002'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudetwo = location.latitude
longitudetwo = location.longitude
print('The geograpical coordinate of 75002 are {}, {}.'.format(latitudetwo, longitudetwo))

address = '75003'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudethree = location.latitude
longitudethree = location.longitude
print('The geograpical coordinate of 75003 are {}, {}.'.format(latitudethree, longitudethree))

address = '75004'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudefour = location.latitude
longitudefour = location.longitude
print('The geograpical coordinate of 75004 are {}, {}.'.format(latitudefour, longitudefour))

address = '75005'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudefive = location.latitude
longitudefive = location.longitude
print('The geograpical coordinate of 75005 are {}, {}.'.format(latitudefive, longitudefive))

address = '75006'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudesix = location.latitude
longitudesix = location.longitude
print('The geograpical coordinate of 75006 are {}, {}.'.format(latitudesix, longitudesix))

address = '75007'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudeseven = location.latitude
longitudeseven = location.longitude
print('The geograpical coordinate of 75007 are {}, {}.'.format(latitudeseven, longitudeseven))

address = '75008'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudeeight = location.latitude
longitudeeight = location.longitude
print('The geograpical coordinate of 75008 are {}, {}.'.format(latitudeeight, longitudeeight))

address = '75009'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudenine = location.latitude
longitudenine = location.longitude
print('The geograpical coordinate of 75009 are {}, {}.'.format(latitudenine, longitudenine))

address = '75010'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudeten = location.latitude
longitudeten = location.longitude
print('The geograpical coordinate of 75010 are {}, {}.'.format(latitudeten, longitudeten))

address = '75011'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudeeleven = location.latitude
longitudeeleven = location.longitude
print('The geograpical coordinate of 75011 are {}, {}.'.format(latitudeeleven, longitudeeleven))

address = '75012'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudetwelve = location.latitude
longitudetwelve = location.longitude
print('The geograpical coordinate of 75012 are {}, {}.'.format(latitudetwelve, longitudetwelve))

address = '75013'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudethirte = location.latitude
longitudethirte = location.longitude
print('The geograpical coordinate of 75013 are {}, {}.'.format(latitudethirte, longitudethirte))

address = '75014'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudefourte = location.latitude
longitudefourte = location.longitude
print('The geograpical coordinate of 75014 are {}, {}.'.format(latitudefourte, longitudefourte))

address = '75015'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudefifte = location.latitude
longitudefifte = location.longitude
print('The geograpical coordinate of 75015 are {}, {}.'.format(latitudefifte, longitudefifte))

address = '75016'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudesixte = location.latitude
longitudesixte = location.longitude
print('The geograpical coordinate of 75016 are {}, {}.'.format(latitudesixte, longitudesixte))

address = '75017'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudesevente = location.latitude
longitudesevente = location.longitude
print('The geograpical coordinate of 75017 are {}, {}.'.format(latitudesevente, longitudesevente))

address = '75018'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudeeighte = location.latitude
longitudeeighte = location.longitude
print('The geograpical coordinate of 75018 are {}, {}.'.format(latitudeeighte, longitudeeighte))

address = '75019'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudeninete = location.latitude
longitudeninete = location.longitude
print('The geograpical coordinate of 75019 are {}, {}.'.format(latitudeninete, longitudeninete))

address = '75020'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitudetwenty = location.latitude
longitudetwenty = location.longitude
print('The geograpical coordinate of 75020 are {}, {}.'.format(latitudetwenty, longitudetwenty))

The geograpical coordinate of 75000 are -16.5467946, -49.2692742.
The geograpical coordinate of 75001 are 48.8635304715812, 2.33893950508195.
The geograpical coordinate of 75002 are 48.8672755917913, 2.34442508383285.
The geograpical coordinate of 75003 are 48.8626401744849, 2.36035142715659.
The geograpical coordinate of 75004 are 48.8559933866481, 2.35703001796235.
The geograpical coordinate of 75005 are 48.8458396691871, 2.34861025857353.
The geograpical coordinate of 75006 are 48.85353725, 2.34337024122443.
The geograpical coordinate of 75007 are 48.855896700173, 2.31369940048725.
The geograpical coordinate of 75008 are 48.8760477, 2.3156587.
The geograpical coordinate of 75009 are 48.8806582, 2.34237184510633.
The geograpical coordinate of 75010 are 48.8792014, 2.3543906.
The geograpical coordinate of 75011 are 48.8587491829173, 2.37782693255992.
The geograpical coordinate of 75012 are 48.8426096368254, 2.38762639345297.
The geograpical coordinate of 75013 are 48.8293667109781, 2.

In [10]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) 

# add 75000 as a red circle mark
folium.features.CircleMarker(
    [latitudezero, longitudezero],
    radius=10,
    popup='75000',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75001 as a red circle mark
folium.features.CircleMarker(
    [latitudeone, longitudeone],
    radius=10,
    popup='75001',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75002 as a red circle mark
folium.features.CircleMarker(
    [latitudetwo, longitudetwo],
    radius=10,
    popup='75002',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75003 as a red circle mark
folium.features.CircleMarker(
    [latitudethree, longitudethree],
    radius=10,
    popup='75003',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75004 as a red circle mark
folium.features.CircleMarker(
    [latitudefour, longitudefour],
    radius=10,
    popup='75004',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75005 as a red circle mark
folium.features.CircleMarker(
    [latitudefive, longitudefive],
    radius=10,
    popup='75005',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75006 as a red circle mark
folium.features.CircleMarker(
    [latitudesix, longitudesix],
    radius=10,
    popup='75006',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75007 as a red circle mark
folium.features.CircleMarker(
    [latitudeseven, longitudeseven],
    radius=10,
    popup='75007',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75008 as a red circle mark
folium.features.CircleMarker(
    [latitudeeight, longitudeeight],
    radius=10,
    popup='75008',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75009 as a red circle mark
folium.features.CircleMarker(
    [latitudenine, longitudenine],
    radius=10,
    popup='75009',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75010 as a red circle mark
folium.features.CircleMarker(
    [latitudeten, longitudeten],
    radius=10,
    popup='75010',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75011 as a red circle mark
folium.features.CircleMarker(
    [latitudeeleven, longitudeeleven],
    radius=10,
    popup='75011',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75012 as a red circle mark
folium.features.CircleMarker(
    [latitudetwelve, longitudetwelve],
    radius=10,
    popup='75012',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75013 as a red circle mark
folium.features.CircleMarker(
    [latitudethirte, longitudethirte],
    radius=10,
    popup='75013',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75014 as a red circle mark
folium.features.CircleMarker(
    [latitudefourte, longitudefourte],
    radius=10,
    popup='75014',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75015 as a red circle mark
folium.features.CircleMarker(
    [latitudefifte, longitudefifte],
    radius=10,
    popup='75015',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75016 as a red circle mark
folium.features.CircleMarker(
    [latitudesixte, longitudesixte],
    radius=10,
    popup='75016',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75017 as a red circle mark
folium.features.CircleMarker(
    [latitudesevente, longitudesevente],
    radius=10,
    popup='75017',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75018 as a red circle mark
folium.features.CircleMarker(
    [latitudeeighte, longitudeeighte],
    radius=10,
    popup='75018',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75019 as a red circle mark
folium.features.CircleMarker(
    [latitudeninete, longitudeninete],
    radius=10,
    popup='75019',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add 75020 as a red circle mark
folium.features.CircleMarker(
    [latitudetwenty, longitudetwenty],
    radius=10,
    popup='75020',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        # Map does not appear if the popup in CircleMarker contains an apostrophe, so I remove apostrophe
        popup=label.replace("'",""),
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

### K-Means Clustering

In [12]:
# A function which compute the great circle distance between gps coordinates of two points  
from geopy.distance import great_circle

In [13]:
column_names = ['name','categories', 'address', 'cc', 'city', 'country','crossStreet','distance','formattedAddress','labeledLatLngs','lat','lng','postalCode','state','id'] 
df_newDistance = pd.DataFrame(columns=column_names)
df_newDistance

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id


In [14]:
df_fourth = dataframe_filtered[dataframe_filtered['postalCode'] == '75004']


# The GPS coordinates of borough '75004' are on variables: latitudefour and longitudefour 
GPSBourough = (latitudefour, longitudefour)

for row in df_fourth.iterrows():
    #row[1][7]=='Not assigned' :
    value1 = (row[1][10], row[1][11])
    row[1][7] = great_circle(GPSBourough, value1).meters
    #print(row[1][7])
    df_newDistance = df_newDistance.append({'name':row[1][0] ,
                                            'categories': row[1][1],
                                            'address': row[1][2],
                                            'cc': row[1][3],
                                            'city': row[1][4], 
                                            'country': row[1][5],
                                            'crossStreet': row[1][6],
                                            'distance': row[1][7],
                                            'formattedAddress': row[1][8], 
                                            'labeledLatLngs': row[1][9],
                                            'lat': row[1][10],
                                            'lng': row[1][11],
                                            'postalCode': row[1][12],
                                            'state': row[1][13],
                                            'id': row[1][14]}, ignore_index=True)


df_newDistance.head(30)


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Place de l'Hôtel de Ville – Esplanade de la Li...,Plaza,Place de l'Hôtel de Ville,FR,Paris,France,NaN,410.770942,"[Place de l'Hôtel de Ville, 75004 Paris, France]",NaN,48.857138,2.351693,75004,Île-de-France,4bf41231e5eba59334341f90
1,Häagen-Dazs,Ice Cream Shop,1 rue d'Arcole,FR,Paris,France,NaN,520.491472,"[1 rue d'Arcole, 75004 Paris, France]","[{'label': 'display', 'lng': 2.350124778904122...",48.854873,2.350125,75004,Île-de-France,4cb2fc20db32f04d6da9d34d
2,LUSH,Cosmetics Shop,3 rue du Renard,FR,Paris,France,NaN,443.416241,"[3 rue du Renard, 75004 Paris, France]","[{'label': 'display', 'lng': 2.351621963273821...",48.857791,2.351622,75004,Île-de-France,4d628e33e4fe548152b8979e
3,L'Alsacien,Alsatian Restaurant,6 rue Saint-Bon,FR,Paris,France,NaN,548.805002,"[6 rue Saint-Bon, 75004 Paris, France]","[{'label': 'display', 'lng': 2.350380611949489...",48.858275,2.350381,75004,Île-de-France,54351b59498e123ee4afc7ff
4,BHV Marais,Department Store,52 rue de Rivoli,FR,Paris,France,Rue de Rivoli,281.910672,"[52 rue de Rivoli (Rue de Rivoli), 75004 Paris...","[{'label': 'display', 'lng': 2.353650749944108...",48.857210,2.353651,75004,Île-de-France,4b4b6a3df964a520b89a26e3
5,Maison Aleph,Pastry Shop,20 rue de la Verrerie,FR,Paris,France,NaN,218.216254,"[20 rue de la Verrerie, 75004 Paris, France]","[{'label': 'display', 'lng': 2.354873, 'lat': ...",48.857348,2.354873,75004,Île-de-France,59247f4e0d2be70573edd834
6,Tour Saint-Jacques,Historic Site,88 rue de Rivoli,FR,Paris,France,NaN,638.365912,"[88 rue de Rivoli, 75004 Paris, France]","[{'label': 'display', 'lng': 2.348874807357788...",48.858031,2.348875,75004,Île-de-France,4b5c7d1ff964a5205f3229e3
7,Paris Rendez-Vous,Souvenir Shop,29 rue de Rivoli,FR,Paris,France,Rue Lobau,332.423414,"[29 rue de Rivoli (Rue Lobau), 75004 Paris, Fr...","[{'label': 'display', 'lng': 2.352841377331694...",48.857150,2.352841,75004,Île-de-France,5361075a498ea34171cb06ca
8,Hotel Duo Paris,Hotel,11 rue du Temple,FR,Paris,France,Rue de la Verrerie,377.893642,"[11 rue du Temple (Rue de la Verrerie), 75004 ...","[{'label': 'display', 'lng': 2.352936699640952...",48.858065,2.352937,75004,Île-de-France,4e84bdeb2fb6611bdfd704ea
9,Au Bougnat,Auvergne Restaurant,26 rue Chanoinesse,FR,Paris,France,NaN,543.304043,"[26 rue Chanoinesse, 75004 Paris, France]","[{'label': 'display', 'lng': 2.350065379156101...",48.854302,2.350065,75004,Île-de-France,4c61c047eb82d13afac003d6


In [15]:
df_cluster = df_newDistance[['lat', 'lng', 'distance']]

In [16]:
# set number of clusters
kclusters = 2

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 1, 1, 0, 0, 1, 0, 0, 1], dtype=int32)

In [17]:
# add clustering labels
df_newDistance.insert(0, 'Cluster Labels', kmeans.labels_)

df_newDistance.head(30)

,Cluster Labels,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,0,Place de l'Hôtel de Ville – Esplanade de la Li...,Plaza,Place de l'Hôtel de Ville,FR,Paris,France,NaN,410.770942,"[Place de l'Hôtel de Ville, 75004 Paris, France]",NaN,48.857138,2.351693,75004,Île-de-France,4bf41231e5eba59334341f90
1,1,Häagen-Dazs,Ice Cream Shop,1 rue d'Arcole,FR,Paris,France,NaN,520.491472,"[1 rue d'Arcole, 75004 Paris, France]","[{'label': 'display', 'lng': 2.350124778904122...",48.854873,2.350125,75004,Île-de-France,4cb2fc20db32f04d6da9d34d
2,1,LUSH,Cosmetics Shop,3 rue du Renard,FR,Paris,France,NaN,443.416241,"[3 rue du Renard, 75004 Paris, France]","[{'label': 'display', 'lng': 2.351621963273821...",48.857791,2.351622,75004,Île-de-France,4d628e33e4fe548152b8979e
3,1,L'Alsacien,Alsatian Restaurant,6 rue Saint-Bon,FR,Paris,France,NaN,548.805002,"[6 rue Saint-Bon, 75004 Paris, France]","[{'label': 'display', 'lng': 2.350380611949489...",48.858275,2.350381,75004,Île-de-France,54351b59498e123ee4afc7ff
4,0,BHV Marais,Department Store,52 rue de Rivoli,FR,Paris,France,Rue de Rivoli,281.910672,"[52 rue de Rivoli (Rue de Rivoli), 75004 Paris...","[{'label': 'display', 'lng': 2.353650749944108...",48.857210,2.353651,75004,Île-de-France,4b4b6a3df964a520b89a26e3
5,0,Maison Aleph,Pastry Shop,20 rue de la Verrerie,FR,Paris,France,NaN,218.216254,"[20 rue de la Verrerie, 75004 Paris, France]","[{'label': 'display', 'lng': 2.354873, 'lat': ...",48.857348,2.354873,75004,Île-de-France,59247f4e0d2be70573edd834
6,1,Tour Saint-Jacques,Historic Site,88 rue de Rivoli,FR,Paris,France,NaN,638.365912,"[88 rue de Rivoli, 75004 Paris, France]","[{'label': 'display', 'lng': 2.348874807357788...",48.858031,2.348875,75004,Île-de-France,4b5c7d1ff964a5205f3229e3
7,0,Paris Rendez-Vous,Souvenir Shop,29 rue de Rivoli,FR,Paris,France,Rue Lobau,332.423414,"[29 rue de Rivoli (Rue Lobau), 75004 Paris, Fr...","[{'label': 'display', 'lng': 2.352841377331694...",48.857150,2.352841,75004,Île-de-France,5361075a498ea34171cb06ca
8,0,Hotel Duo Paris,Hotel,11 rue du Temple,FR,Paris,France,Rue de la Verrerie,377.893642,"[11 rue du Temple (Rue de la Verrerie), 75004 ...","[{'label': 'display', 'lng': 2.352936699640952...",48.858065,2.352937,75004,Île-de-France,4e84bdeb2fb6611bdfd704ea
9,1,Au Bougnat,Auvergne Restaurant,26 rue Chanoinesse,FR,Paris,France,NaN,543.304043,"[26 rue Chanoinesse, 75004 Paris, France]","[{'label': 'display', 'lng': 2.350065379156101...",48.854302,2.350065,75004,Île-de-France,4c61c047eb82d13afac003d6


In [18]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add the center of 75004 as a yellow circle mark
LatFourcenter = 48.855993386648
LngFourcenter = 2.3570300179623
folium.features.CircleMarker(
    [LatFourcenter, LngFourcenter],
    radius=10,
    popup='75004',
    fill=True,
    color='yellow',
    fill_color='yellow',
    fill_opacity=0.6
    ).add_to(map_clusters)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_newDistance['lat'], df_newDistance['lng'], df_newDistance['categories'], df_newDistance['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [21]:
# Showing the nearest venues from the center of 75004
df_newDistance.loc[df_newDistance['Cluster Labels'] == 0]

,Cluster Labels,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,0,Place de l'Hôtel de Ville – Esplanade de la Li...,Plaza,Place de l'Hôtel de Ville,FR,Paris,France,NaN,410.770942,"[Place de l'Hôtel de Ville, 75004 Paris, France]",NaN,48.857138,2.351693,75004,Île-de-France,4bf41231e5eba59334341f90
4,0,BHV Marais,Department Store,52 rue de Rivoli,FR,Paris,France,Rue de Rivoli,281.910672,"[52 rue de Rivoli (Rue de Rivoli), 75004 Paris...","[{'label': 'display', 'lng': 2.353650749944108...",48.857210,2.353651,75004,Île-de-France,4b4b6a3df964a520b89a26e3
5,0,Maison Aleph,Pastry Shop,20 rue de la Verrerie,FR,Paris,France,NaN,218.216254,"[20 rue de la Verrerie, 75004 Paris, France]","[{'label': 'display', 'lng': 2.354873, 'lat': ...",48.857348,2.354873,75004,Île-de-France,59247f4e0d2be70573edd834
7,0,Paris Rendez-Vous,Souvenir Shop,29 rue de Rivoli,FR,Paris,France,Rue Lobau,332.423414,"[29 rue de Rivoli (Rue Lobau), 75004 Paris, Fr...","[{'label': 'display', 'lng': 2.352841377331694...",48.857150,2.352841,75004,Île-de-France,5361075a498ea34171cb06ca
8,0,Hotel Duo Paris,Hotel,11 rue du Temple,FR,Paris,France,Rue de la Verrerie,377.893642,"[11 rue du Temple (Rue de la Verrerie), 75004 ...","[{'label': 'display', 'lng': 2.352936699640952...",48.858065,2.352937,75004,Île-de-France,4e84bdeb2fb6611bdfd704ea
10,0,Fleux',Furniture / Home Store,39 et 52 rue Sainte-Croix de la Bretonnerie,FR,Paris,France,NaN,383.704403,"[39 et 52 rue Sainte-Croix de la Bretonnerie, ...","[{'label': 'display', 'lng': 2.354148718454883...",48.858876,2.354149,75004,Île-de-France,4b533b45f964a520509327e3
11,0,BHV Marais – L'Homme,Men's Store,36 rue de la Verrerie,FR,Paris,France,NaN,287.842409,"[36 rue de la Verrerie, 75004 Paris, France]","[{'label': 'display', 'lng': 2.353649716361928...",48.857317,2.353650,75004,Île-de-France,4be150de0365c9b6bc4cb338
12,0,Yummy and Guiltfree,Gluten-free Restaurant,3 rue du Temple,FR,Paris,France,NaN,365.441743,"[3 rue du Temple, 75004 Paris, France]","[{'label': 'display', 'lng': 2.352604759342985...",48.857516,2.352605,75004,Île-de-France,5901dcce5d891b426af3e948
13,0,Les Mauvais Garçons,Lyonese Bouchon,4 rue des Mauvais Garçons,FR,Paris,France,NaN,170.060191,"[4 rue des Mauvais Garçons, 75004 Paris, France]","[{'label': 'display', 'lng': 2.35508376610984,...",48.856829,2.355084,75004,Île-de-France,4bc9d039937ca5937f57a692
17,0,The Body Shop,Cosmetics Shop,68 rue de Rivoli,FR,Paris,France,NaN,398.860220,"[68 rue de Rivoli, 75004 Paris, France]","[{'label': 'display', 'lng': 2.35210557, 'lat'...",48.857530,2.352106,75004,Île-de-France,4d99bbbadaec224bd286653e


In [22]:
# Showing the farthest venues from the center of 75004
df_newDistance.loc[df_newDistance['Cluster Labels'] == 1]

,Cluster Labels,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
1,1,Häagen-Dazs,Ice Cream Shop,1 rue d'Arcole,FR,Paris,France,NaN,520.491472,"[1 rue d'Arcole, 75004 Paris, France]","[{'label': 'display', 'lng': 2.350124778904122...",48.854873,2.350125,75004,Île-de-France,4cb2fc20db32f04d6da9d34d
2,1,LUSH,Cosmetics Shop,3 rue du Renard,FR,Paris,France,NaN,443.416241,"[3 rue du Renard, 75004 Paris, France]","[{'label': 'display', 'lng': 2.351621963273821...",48.857791,2.351622,75004,Île-de-France,4d628e33e4fe548152b8979e
3,1,L'Alsacien,Alsatian Restaurant,6 rue Saint-Bon,FR,Paris,France,NaN,548.805002,"[6 rue Saint-Bon, 75004 Paris, France]","[{'label': 'display', 'lng': 2.350380611949489...",48.858275,2.350381,75004,Île-de-France,54351b59498e123ee4afc7ff
6,1,Tour Saint-Jacques,Historic Site,88 rue de Rivoli,FR,Paris,France,NaN,638.365912,"[88 rue de Rivoli, 75004 Paris, France]","[{'label': 'display', 'lng': 2.348874807357788...",48.858031,2.348875,75004,Île-de-France,4b5c7d1ff964a5205f3229e3
9,1,Au Bougnat,Auvergne Restaurant,26 rue Chanoinesse,FR,Paris,France,NaN,543.304043,"[26 rue Chanoinesse, 75004 Paris, France]","[{'label': 'display', 'lng': 2.350065379156101...",48.854302,2.350065,75004,Île-de-France,4c61c047eb82d13afac003d6
14,1,Au Vieux Paris d'Arcole,French Restaurant,24 rue Chanoinesse,FR,Paris,France,NaN,530.717509,"[24 rue Chanoinesse, 75004 Paris, France]","[{'label': 'display', 'lng': 2.350312471389770...",48.854196,2.350312,75004,Île-de-France,4b9ea5cdf964a52094f636e3
15,1,Théâtre de la Ville,Theater,2 place du Châtelet,FR,Paris,France,NaN,681.568152,"[2 place du Châtelet, 75004 Paris, France]","[{'label': 'display', 'lng': 2.347911117453113...",48.857239,2.347911,75004,Île-de-France,4adcda15f964a520a13721e3
16,1,Yogurt Factory,Frozen Yogurt Shop,3 rue Saint-Merri,FR,Paris,France,Rue du Temple,426.131563,"[3 rue Saint-Merri (Rue du Temple), 75004 Pari...","[{'label': 'display', 'lng': 2.353587175456823...",48.859083,2.353587,75004,Île-de-France,4ff07da4e4b0b61f2a827eb3
19,1,Bagel Baget Cafe,Bagel Shop,91 rue de la Verrerie,FR,Paris,France,NaN,554.020782,"[91 rue de la Verrerie, 75004 Paris, France]","[{'label': 'display', 'lng': 2.350769489560218...",48.858794,2.350769,75004,Île-de-France,53d13101498e918e659f88a2
20,1,Huré,Bakery,1 rue d'Arcole,FR,Paris,France,NaN,522.786978,"[1 rue d'Arcole, 75004 Paris, France]","[{'label': 'display', 'lng': 2.350112033392268...",48.854821,2.350112,75004,Île-de-France,4f5f2ebee4b033732bc81ff9
